In [1]:
import pandas as pd
import numpy as np
import math
import scipy.stats as stats
import json
import requests

from functools import reduce
import yfinance as yf

import datetime as dt
from datetime import datetime, date, time, timedelta
from dateutil.relativedelta import relativedelta

# Import PlotLy Dependencies
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
%run data_source.ipynb

In [3]:
# CSUSHPISA - S&P/Case-Shiller U.S. National Home Price Index
case_schiller_hpi_df = pd.DataFrame(fred_api_function("CSUSHPISA"))

# Rename the 'Value' column to 'Case Shiller HPI'
case_schiller_hpi_df.rename(columns={'Value': 'Case Shiller HPI'}, inplace=True)

case_schiller_hpi_df

,Date,Case Shiller HPI
0,1987-01-01,63.965
1,1987-02-01,64.424
2,1987-03-01,64.736
3,1987-04-01,65.132
4,1987-05-01,65.563
...,...,...
436,2023-05-01,302.566
437,2023-06-01,304.593
438,2023-07-01,306.767
439,2023-08-01,309.155


In [4]:
# USSTHPI - All-Transactions House Price Index for the United States
fhfa_hpi_df = pd.DataFrame(fred_api_function("USSTHPI"))

# Rename the 'Value' column to 'FHFA HPI'
fhfa_hpi_df.rename(columns={'Value': 'FHFA HPI'}, inplace=True)

fhfa_hpi_df

,Date,FHFA HPI
0,1975-01-01,60.06
1,1975-04-01,61.01
2,1975-07-01,61.18
3,1975-10-01,62.24
4,1976-01-01,62.87
...,...,...
190,2022-07-01,627.42
191,2022-10-01,622.78
192,2023-01-01,625.78
193,2023-04-01,645.38


In [21]:
# HOEREPHRE - Households; Owners' Equity in Real Estate as a Percentage of Household Real Estate
homeowners_equity_df = pd.DataFrame(fred_api_function("HOEREPHRE"))

# Rename the 'Value' column to 'Homeowners Owners Equity'
homeowners_equity_df.rename(columns={'Value': 'Homeowners Owners Equity (%)'}, inplace=True)

homeowners_equity_df

,Date,Homeowners Owners Equity (%)
0,1945-10-01,83.880947
1,1946-10-01,82.686513
2,1947-10-01,84.072507
3,1948-10-01,83.232494
4,1949-10-01,82.725195
...,...,...
288,2022-04-01,72.312319
289,2022-07-01,70.993280
290,2022-10-01,70.011113
291,2023-01-01,69.668083


In [22]:
real_estate_df = pd.DataFrame()

# Merge DataFrames on the 'Date' column
real_estate_df = pd.merge(case_schiller_hpi_df, fhfa_hpi_df, on='Date').merge(homeowners_equity_df, on='Date')

real_estate_df

,Date,Case Shiller HPI,FHFA HPI,Homeowners Owners Equity (%)
0,1987-01-01,63.965,142.44,69.487954
1,1987-04-01,65.132,144.66,69.005579
2,1987-07-01,66.507,146.33,68.860061
3,1987-10-01,67.739,147.28,68.935681
4,1988-01-01,68.858,149.57,68.899341
...,...,...,...,...
141,2022-04-01,300.573,617.03,72.312319
142,2022-07-01,303.879,627.42,70.993280
143,2022-10-01,298.873,622.78,70.011113
144,2023-01-01,297.030,625.78,69.668083


In [28]:
# Normalize Data Scaling
norm_housing_df = pd.DataFrame()

# Calculate Maximum and Minimums for each feature
case_shiller_min = real_estate_df['Case Shiller HPI'].min()
case_shiller_max = real_estate_df['Case Shiller HPI'].max()
fhfa_min = real_estate_df['FHFA HPI'].min()
fhfa_max = real_estate_df['FHFA HPI'].max()
equity_min = real_estate_df['Homeowners Owners Equity (%)'].min()
equity_max = real_estate_df['Homeowners Owners Equity (%)'].max()

# Calculate Normalization of each feature
norm_housing_df["Date"] = real_estate_df['Date']
norm_housing_df["Normalized Case Shiller Index"] = (real_estate_df['Case Shiller HPI'] - case_shiller_min) / (case_shiller_max - case_shiller_min)
norm_housing_df["Normalized FHFA HPI"] = (real_estate_df['FHFA HPI'] - fhfa_min) / (fhfa_max - fhfa_min)
norm_housing_df["Normalized Homeowners Owners Equity (%)"] = (real_estate_df['Homeowners Owners Equity (%)'] - equity_min) / (equity_max - equity_min)

# How each feature contributes to the Housing Index needs to be weighted to provide a single number for the Housing Index contribution to the algorithm
# Note:- The corresponding weights are 20%, 20%, and 60% results in the total of 100%
norm_housing_df["Overall Housing Index"] = (norm_housing_df["Normalized Case Shiller Index"] * 0.20) +    \
                                                (norm_housing_df["Normalized FHFA HPI"] * 0.20) +  \
                                                (norm_housing_df["Normalized Homeowners Owners Equity (%)"] * (0.60))

# Overall Index started at 0. Drop the first row for the change dataframe
norm_housing_df = norm_housing_df.drop(0).reset_index(drop=True)

# %store norm_housing_df
norm_housing_df

,Date,Normalized Case Shiller Index,Normalized FHFA HPI,Normalized Homeowners Owners Equity (%),Overall Housing Index
0,1987-04-01,0.004864,0.004414,0.874905,0.526798
1,1987-07-01,0.010595,0.007735,0.869400,0.525306
2,1987-10-01,0.015731,0.009623,0.872260,0.528427
3,1988-01-01,0.020395,0.014177,0.870886,0.529446
4,1988-04-01,0.025059,0.020142,0.865157,0.528134
...,...,...,...,...,...
140,2022-04-01,0.986220,0.943631,1.000000,0.985970
141,2022-07-01,1.000000,0.964290,0.950100,0.962918
142,2022-10-01,0.979134,0.955064,0.912944,0.934606
143,2023-01-01,0.971452,0.961029,0.899967,0.926477


In [29]:
# Plot Resulting Housing Data

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = norm_housing_df['Date'],
    y = norm_housing_df['Normalized Case Shiller Index'],
    line = dict(color='blue', width=1, dash='dot'),
    showlegend=True,
    name="Normalized Case Shiller Index"
))

fig.add_trace(go.Scatter(
    x = norm_housing_df['Date'],
    y = norm_housing_df['Normalized FHFA HPI'],
    line = dict(color='red', width=1, dash='dot'),
    showlegend=True,
    name="Normalized FHFA HPI"
))
fig.add_trace(go.Scatter(
    x = norm_housing_df['Date'],
    y = norm_housing_df['Normalized Homeowners Owners Equity (%)'],
    line = dict(color='green', width=1, dash='dot'),
    showlegend=True,
    name="Normalized Homeowners Owners Equity (%)"
))
fig.add_trace(go.Scatter(
    x = norm_housing_df['Date'],
    y = norm_housing_df['Overall Housing Index'],
    line = dict(color='black', width=2),
    showlegend=True,
    name="Overall Housing Index"
))

fig.update_layout(
    title="Overall Housing Index"
)


In [33]:
# Plot Overall Housing Index Change
norm_housing_change_df = pd.DataFrame()

norm_housing_change_df["Date"] = norm_housing_df["Date"]
norm_housing_change_df["Overall Housing Index"] = norm_housing_df["Overall Housing Index"]
norm_housing_change_df["Change in Overall Housing Index (%)"] = ((norm_housing_df["Overall Housing Index"] -  \
                                                        norm_housing_df["Overall Housing Index"].shift(1)) /  \
                                                        norm_housing_df["Overall Housing Index"]) * 100

# Calculate Change in Velocity
norm_housing_change_df["Housing Change in Velocity (basis points)"] = (norm_housing_change_df["Change in Overall Housing Index (%)"] -  \
                                                    norm_housing_change_df["Change in Overall Housing Index (%)"].shift(1)) * 100
norm_housing_change_df

norm_housing_change_df

,Date,Overall Housing Index,Change in Overall Housing Index (%),Housing Change in Velocity (basis points)
0,1987-04-01,0.526798,NaN,NaN
1,1987-07-01,0.525306,-0.284153,NaN
2,1987-10-01,0.528427,0.590669,87.482221
3,1988-01-01,0.529446,0.192398,-39.827122
4,1988-04-01,0.528134,-0.248314,-44.071145
...,...,...,...,...
140,2022-04-01,0.985970,4.742350,-277.193408
141,2022-07-01,0.962918,-2.393995,-713.634527
142,2022-10-01,0.934606,-3.029276,-63.528110
143,2023-01-01,0.926477,-0.877472,215.180349


In [34]:
# Function to convert quarterly data to monthly data for Overall Housing Index

def convert_to_monthly(df):
    
    monthly_df = pd.DataFrame()

    date = []
    index = []
    index_change = []
    velocity_change = []
    
    for i, row in df.iterrows():
        
        original_date = pd.to_datetime(row[0])
        date.append(original_date)
        index.append(row[1])
        index_change.append(row[2])
        velocity_change.append(row[3])
    
        date_plus_1 = original_date + pd.DateOffset(months=1)
        date.append(date_plus_1)
        index.append(row[1])
        index_change.append(row[2])
        velocity_change.append(row[3])
        
        date_plus_2 = original_date + pd.DateOffset(months=2)
        date.append(date_plus_2)
        index.append(row[1])
        index_change.append(row[2])
        velocity_change.append(row[3])
        
    data = {'Date': date,
            'Overall Housing Index': index,
            'Change in Overall Housing Index (%)': index_change,
            'Housing Change in Velocity (basis points)': velocity_change}
    
    monthly_df = pd.DataFrame(data)
    return monthly_df

housing_monthly_data = convert_to_monthly(norm_housing_change_df)


housing_monthly_data

,Date,Overall Housing Index,Change in Overall Housing Index (%),Housing Change in Velocity (basis points)
0,1987-04-01,0.526798,NaN,NaN
1,1987-05-01,0.526798,NaN,NaN
2,1987-06-01,0.526798,NaN,NaN
3,1987-07-01,0.525306,-0.284153,NaN
4,1987-08-01,0.525306,-0.284153,NaN
...,...,...,...,...
430,2023-02-01,0.926477,-0.877472,215.180349
431,2023-03-01,0.926477,-0.877472,215.180349
432,2023-04-01,0.969956,4.482617,536.008947
433,2023-05-01,0.969956,4.482617,536.008947


In [35]:
# Store dataframe to export
%store housing_monthly_data

Stored 'housing_monthly_data' (DataFrame)


In [36]:
# Plot Resulting Housing Data

fig = go.Figure()
fig.add_trace(go.Scatter(
    x = housing_monthly_data['Date'],
    y = housing_monthly_data['Housing Change in Velocity (basis points)'],
    line = dict(color='blue', width=1),
    showlegend=False
))

fig.update_layout(
    title="Housing Change in Velocity (basis points)"
)
fig.update_yaxes(
    title_text="basis points"
)